In [1]:
import pandas as pd 
import numpy as np

In [2]:
train_val_df_raw = pd.read_csv('/Users/thananpornsethjinda/Desktop/internship/projects/titanic-survival-prediction/data/train.csv')
test_df_raw = pd.read_csv('/Users/thananpornsethjinda/Desktop/internship/projects/titanic-survival-prediction/data/test.csv')

In [3]:
from basic_cleaning import clean_dataframe

train_val_df = clean_dataframe(train_val_df_raw)

print("-" * 100)

test_df = clean_dataframe(test_df_raw)

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age         177
cabin       687
embarked      2
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket', 'cabin']
[INFO] Potential numeric outliers detected:
{'age': np.int64(11), 'sibsp': np.int64(46), 'parch': np.int64(213), 'fare': np.int64(116)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.
----------------------------------------------------------------------------------------------------
[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age       86
fare       1
cabin    327
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket']
[INFO] Potential numeric outliers detected:
{'age': np.int64(2), 'sibsp': np.int64(11), 'parch': np.int64(94), 'fare': n

In [4]:
combined = pd.concat([train_val_df, test_df])

In [5]:
train_val_df

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"braund, mr. owen harris",male,22.0,1,0,a/5 21171,7.2500,NaN,s
1,2,1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,pc 17599,71.2833,c85,c
2,3,1,3,"heikkinen, miss. laina",female,26.0,0,0,ston/o2. 3101282,7.9250,NaN,s
3,4,1,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,113803,53.1000,c123,s
4,5,0,3,"allen, mr. william henry",male,35.0,0,0,373450,8.0500,NaN,s
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"montvila, rev. juozas",male,27.0,0,0,211536,13.0000,NaN,s
887,888,1,1,"graham, miss. margaret edith",female,19.0,0,0,112053,30.0000,b42,s
888,889,0,3,"johnston, miss. catherine helen ""carrie""",female,NaN,1,2,w./c. 6607,23.4500,NaN,s
889,890,1,1,"behr, mr. karl howell",male,26.0,0,0,111369,30.0000,c148,c


In [6]:
len(train_val_df['ticket'].unique())

681

In [7]:
train_val_df['ticket'].value_counts()

ticket
347082      7
ca. 2343    7
1601        7
3101295     6
ca 2144     6
           ..
9234        1
19988       1
2693        1
pc 17612    1
370376      1
Name: count, Length: 681, dtype: int64

In [8]:
train_val_df.loc[train_val_df['ticket'] == '347082']

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
13,14,0,3,"andersson, mr. anders johan",male,39.0,1,5,347082,31.275,NaN,s
119,120,0,3,"andersson, miss. ellis anna maria",female,2.0,4,2,347082,31.275,NaN,s
541,542,0,3,"andersson, miss. ingeborg constanzia",female,9.0,4,2,347082,31.275,NaN,s
542,543,0,3,"andersson, miss. sigrid elisabeth",female,11.0,4,2,347082,31.275,NaN,s
610,611,0,3,"andersson, mrs. anders johan (alfrida konstant...",female,39.0,1,5,347082,31.275,NaN,s
813,814,0,3,"andersson, miss. ebba iris alfrida",female,6.0,4,2,347082,31.275,NaN,s
850,851,0,3,"andersson, master. sigvard harald elias",male,4.0,4,2,347082,31.275,NaN,s


In [9]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

In [10]:
X_train, y_train = train_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']], train_df['survived']
X_val, y_val = val_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']], val_df['survived']
X_test = test_df[['pclass', 'sex', 'age', 'embarked','sibsp','parch','name','ticket','fare']]

In [11]:
X_train.groupby('pclass')['age'].mean() 
# you could technically impute the average age for each class instead of imputing one mean for every missing value

pclass
1    37.891049
2    29.843972
3    25.162917
Name: age, dtype: float64

In [12]:
TitleDict = {"capt": "officer","col": "officer","major": "officer","jonkheer": "royalty", \
             "don": "royalty", "sir" : "royalty","dr": "royalty","rev": "royalty", \
             "countess":"royalty", "mme": "mrs", "mlle": "miss", "ms": "mrs","mr" : "mr", \
             "mrs" : "mrs","miss" : "miss","master" : "master","lady" : "royalty"}

def preprocessing(df): 

    df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)

    df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())

    df['ticket_per_person'] = df['fare']/df['ticket_people']

    df['title'] = df.title.map(TitleDict)

    df.loc[(df.title == 'miss') & (df.parch != 0) & (df.ticket_people > 1), 'title'] = "female_child"

    return df


In [13]:
X_train_2 = preprocessing(X_train)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_3578/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)
/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_3578/2284188230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())


In [14]:
X_val_2 = preprocessing(X_val)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_3578/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)
/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_3578/2284188230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticket_people'] = df['ticket'].map(combined['ticket'].value_counts())


In [15]:
X_test_2 = preprocessing(X_test)

/var/folders/19/sr0vg3y90ns0sp5j74zth4h80000gn/T/ipykernel_3578/2284188230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.name.str.extract(r' ([A-Za-z]+)\.', expand=False)


In [16]:
X_test_2.at[414, 'title'] = 'royalty'

In [22]:
grouped = X_train_2.groupby(['pclass', 'sex', 'title'], observed=True)['age'].agg({'mean','median','count'})


In [23]:
grouped

median       mean  count
pclass sex    title                                 
1      female female_child   21.50  20.400000     10
              miss           33.00  34.000000     25
              mrs            39.00  40.033333     30
              royalty        40.50  40.500000      2
       male   master          2.46   2.460000      2
              mr             38.50  40.431818     66
              officer        56.00  56.600000      5
              royalty        44.00  42.000000      3
2      female female_child    6.00  10.000000     10
              miss           29.00  30.593750     16
              mrs            30.50  33.031250     32
       male   master          1.50   2.437500      8
              mr             30.00  32.695652     69
              royalty        52.50  47.666667      6
3      female female_child    4.00   6.452381     21
              miss           21.50  22.041667     36
              mrs            31.00  33.200000     25
       male   master          4.00   5.020000     21
              mr             27.00  29.094595    185

In [32]:
grp = X_train_2.groupby(['pclass', 'sex', 'title'], observed=True)['age'].mean().reset_index()[['pclass', 'sex', 'title', 'age']]

In [33]:
grp

,pclass,sex,title,age
0,1,female,female_child,20.400000
1,1,female,miss,34.000000
2,1,female,mrs,40.033333
3,1,female,royalty,40.500000
4,1,male,master,2.460000
5,1,male,mr,40.431818
6,1,male,officer,56.600000
7,1,male,royalty,42.000000
8,2,female,female_child,10.000000
9,2,female,miss,30.593750


In [37]:
def fill_age(x): 
    return grp[(grp.sex == x.sex) & (grp.pclass == x.pclass) & (grp.title == x.title)]['age'].values[0]


In [40]:
X_train_2['age'] = X_train_2.apply(lambda x: fill_age(x) if np.isnan(x['age']) else x['age'], axis = 1)

In [42]:
X_train_2

,pclass,sex,age,embarked,sibsp,parch,name,ticket,fare,title,ticket_people,ticket_per_person
331,1,male,45.500000,s,0,0,"partner, mr. austen",113043,28.5000,mr,1,28.500000
733,2,male,23.000000,s,0,0,"berriman, mr. william john",28425,13.0000,mr,1,13.000000
382,3,male,32.000000,s,0,0,"tikkanen, mr. juho",ston/o 2. 3101293,7.9250,mr,1,7.925000
704,3,male,26.000000,s,1,0,"hansen, mr. henrik juul",350025,7.8542,mr,1,7.854200
813,3,female,6.000000,s,4,2,"andersson, miss. ebba iris alfrida",347082,31.2750,female_child,7,4.467857
...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,female,21.000000,s,0,0,"salkjelsvik, miss. anna kristine",343120,7.6500,miss,1,7.650000
270,1,male,40.431818,s,0,0,"cairns, mr. alexander",113798,31.0000,mr,2,15.500000
860,3,male,41.000000,s,2,0,"hansen, mr. claus peter",350026,14.1083,mr,2,7.054150
435,1,female,14.000000,s,1,2,"carter, miss. lucile polk",113760,120.0000,female_child,4,30.000000


In [62]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='most_frequent')

X_train_2[['embarked']] = impute.fit_transform(X_train_2[['embarked']])

X_train_2

,pclass,sex,age,embarked,sibsp,parch,name,ticket,fare,title,ticket_people,ticket_per_person
331,1,male,45.500000,s,0,0,"partner, mr. austen",113043,28.5000,mr,1,28.500000
733,2,male,23.000000,s,0,0,"berriman, mr. william john",28425,13.0000,mr,1,13.000000
382,3,male,32.000000,s,0,0,"tikkanen, mr. juho",ston/o 2. 3101293,7.9250,mr,1,7.925000
704,3,male,26.000000,s,1,0,"hansen, mr. henrik juul",350025,7.8542,mr,1,7.854200
813,3,female,6.000000,s,4,2,"andersson, miss. ebba iris alfrida",347082,31.2750,female_child,7,4.467857
...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,female,21.000000,s,0,0,"salkjelsvik, miss. anna kristine",343120,7.6500,miss,1,7.650000
270,1,male,40.431818,s,0,0,"cairns, mr. alexander",113798,31.0000,mr,2,15.500000
860,3,male,41.000000,s,2,0,"hansen, mr. claus peter",350026,14.1083,mr,2,7.054150
435,1,female,14.000000,s,1,2,"carter, miss. lucile polk",113760,120.0000,female_child,4,30.000000


In [ ]:
X_test_2['age'] = X_test_2.apply(lambda x: fill_age(x) if np.isnan(x['age']) else x['age'], axis=1)

In [70]:
X_test_2.loc[:, ['fare', 'ticket_per_person']] = X_train_2['ticket_per_person'].mean()

In [71]:
X_test_2

,pclass,sex,age,embarked,sibsp,parch,name,ticket,fare,title,ticket_people,ticket_per_person
0,3,male,34.500000,q,0,0,"kelly, mr. james",330911,14.351287,mr,1,14.351287
1,3,female,47.000000,s,1,0,"wilkes, mrs. james (ellen needs)",363272,14.351287,mrs,1,14.351287
2,2,male,62.000000,q,0,0,"myles, mr. thomas francis",240276,14.351287,mr,1,14.351287
3,3,male,27.000000,s,0,0,"wirz, mr. albert",315154,14.351287,mr,1,14.351287
4,3,female,22.000000,s,1,1,"hirvonen, mrs. alexander (helga e lindqvist)",3101298,14.351287,mrs,2,14.351287
...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,male,29.094595,s,0,0,"spector, mr. woolf",a.5. 3236,14.351287,mr,1,14.351287
414,1,female,39.000000,c,0,0,"oliva y ocana, dona. fermina",pc 17758,14.351287,royalty,3,14.351287
415,3,male,38.500000,s,0,0,"saether, mr. simon sivertsen",soton/o.q. 3101262,14.351287,mr,1,14.351287
416,3,male,29.094595,s,0,0,"ware, mr. frederick",359309,14.351287,mr,1,14.351287


In [55]:
X_val_2['age'] = X_val_2.apply(lambda x: fill_age(x) if np.isnan(x['age']) else x['age'], axis=1)

In [75]:
def drop_columns(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    return df.drop(['name','ticket','fare','title'], axis=1)
    

In [78]:
X_train_3 = drop_columns(X_train_2)

X_val_3 = drop_columns(X_val_2)

X_test_3 = drop_columns(X_test_2)

In [79]:
X_train_3

,pclass,sex,age,embarked,sibsp,parch,ticket_people,ticket_per_person
331,1,male,45.500000,s,0,0,1,28.500000
733,2,male,23.000000,s,0,0,1,13.000000
382,3,male,32.000000,s,0,0,1,7.925000
704,3,male,26.000000,s,1,0,1,7.854200
813,3,female,6.000000,s,4,2,7,4.467857
...,...,...,...,...,...,...,...,...
106,3,female,21.000000,s,0,0,1,7.650000
270,1,male,40.431818,s,0,0,2,15.500000
860,3,male,41.000000,s,2,0,2,7.054150
435,1,female,14.000000,s,1,2,4,30.000000


In [ ]:
NUMERICAL_COLUMNS = X_train_3.select_dtypes(include=np.number).columns.tolist()

CATEGORICAL_COLUMNS = X_train_3.select_dtypes(include=('object', 'category')).columns.tolist()

In [88]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

enc.fit(X_train_3[CATEGORICAL_COLUMNS])

ENCODED_COLUMNS = list(enc.get_feature_names_out(CATEGORICAL_COLUMNS))

X_train_3[ENCODED_COLUMNS] = enc.transform(X_train_3[CATEGORICAL_COLUMNS]).toarray()

X_val_3[ENCODED_COLUMNS] = enc.transform(X_val_3[CATEGORICAL_COLUMNS]).toarray()

X_test_3[ENCODED_COLUMNS] = enc.transform(X_test_3[CATEGORICAL_COLUMNS]).toarray()

In [90]:
X_train_3[NUMERICAL_COLUMNS + ENCODED_COLUMNS]

,pclass,age,sibsp,parch,ticket_people,ticket_per_person,sex_female,sex_male,embarked_c,embarked_q,embarked_s
331,1,45.500000,0,0,1,28.500000,0.0,1.0,0.0,0.0,1.0
733,2,23.000000,0,0,1,13.000000,0.0,1.0,0.0,0.0,1.0
382,3,32.000000,0,0,1,7.925000,0.0,1.0,0.0,0.0,1.0
704,3,26.000000,1,0,1,7.854200,0.0,1.0,0.0,0.0,1.0
813,3,6.000000,4,2,7,4.467857,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
106,3,21.000000,0,0,1,7.650000,1.0,0.0,0.0,0.0,1.0
270,1,40.431818,0,0,2,15.500000,0.0,1.0,0.0,0.0,1.0
860,3,41.000000,2,0,2,7.054150,0.0,1.0,0.0,0.0,1.0
435,1,14.000000,1,2,4,30.000000,1.0,0.0,0.0,0.0,1.0


In [101]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

X_train_3[NUMERICAL_COLUMNS[1:]] = scale.fit_transform(X_train_3[NUMERICAL_COLUMNS[1:]])

X_val_3[NUMERICAL_COLUMNS[1:]] = scale.transform(X_val_3[NUMERICAL_COLUMNS[1:]])

X_test_3[NUMERICAL_COLUMNS[1:]] = scale.transform(X_test_3[NUMERICAL_COLUMNS[1:]])





In [102]:
X_train_3

,pclass,sex,age,embarked,sibsp,parch,ticket_people,ticket_per_person,sex_female,sex_male,embarked_c,embarked_q,embarked_s
331,1,male,1.201062,s,-0.470722,-0.479342,-0.627383,1.027209,0.0,1.0,0.0,0.0,1.0
733,2,male,-0.473344,s,-0.470722,-0.479342,-0.627383,-0.098105,0.0,1.0,0.0,0.0,1.0
382,3,male,0.196418,s,-0.470722,-0.479342,-0.627383,-0.466554,0.0,1.0,0.0,0.0,1.0
704,3,male,-0.250090,s,0.379923,-0.479342,-0.627383,-0.471694,0.0,1.0,0.0,0.0,1.0
813,3,female,-1.738451,s,2.931860,2.048742,2.613464,-0.717546,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,female,-0.622180,s,-0.470722,-0.479342,-0.627383,-0.486519,1.0,0.0,0.0,0.0,1.0
270,1,male,0.823898,s,-0.470722,-0.479342,-0.087242,0.083398,0.0,1.0,0.0,0.0,1.0
860,3,male,0.866181,s,1.230569,-0.479342,-0.087242,-0.529779,0.0,1.0,0.0,0.0,1.0
435,1,female,-1.143107,s,0.379923,2.048742,0.993040,1.136111,1.0,0.0,0.0,0.0,1.0


In [103]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

ldr = LinearDiscriminantAnalysis()

ldr.fit(X_train_3[NUMERICAL_COLUMNS + ENCODED_COLUMNS], y_train)

LinearDiscriminantAnalysis()

In [104]:
predictions = ldr.predict(X_test_3[NUMERICAL_COLUMNS + ENCODED_COLUMNS])

In [105]:
predictions

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

ideas for feature engineering: 
1. create family size 
2. children present? i dont think so because this column wont work out for children themsevles 
3. the cabin section (A,B,C,D,E and all that) - apparently more than 70% of the data is missing so imputing would just create more noise and possible inaccuracies 
4. extract the title and p-class, and group by title and pclass and find the mean for age 

how are we going find family size - we can just add SibSp and Parch 

eda on fare and survival rate 

eda on embarked and survival rate 

In [106]:

output = pd.DataFrame({'PassengerId': test_df_raw.PassengerId, 'Survived': predictions})



In [97]:
output.isnull().sum()

PassengerId    0
Survived       0
dtype: int64

In [107]:

output.to_csv('submission4.csv', index=False)